In [1]:
import numpy as np
import simpy

# Generate sequence

In [2]:
Machine_num = 3
Machine_label = []
for i in range(0,Machine_num):
    Machine_label.append(i)
print("labels for each machine:\n", Machine_label)

labels for each machine:
 [0, 1, 2]


In [3]:
Product_num = 3
print("The total types of products:\n", Product_num)

The total types of products:
 3


In [4]:
inpro_seq = np.random.randint(50,100)
print("The numbers of products in the sequence:",inpro_seq)

The numbers of products in the sequence: 84


In [5]:
S = []
for i in range(inpro_seq):
    S.append(np.random.randint(0,Product_num))
print("Input Sequence = \n",S)

Input Sequence = 
 [2, 1, 2, 2, 0, 2, 2, 2, 1, 1, 1, 1, 2, 0, 2, 2, 2, 2, 2, 1, 0, 1, 1, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 1, 1, 0, 2, 1, 2, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 1, 0, 0, 0, 0, 2, 0, 2, 1, 0, 1, 2, 0, 0, 1, 1, 1, 2, 1, 1, 2, 0, 0]


In [6]:
# The processsing time for each machine
# The processing time ranges from 10-30
Time_pcs=[]
for i in range(Machine_num):
    Time_pcs.append(np.random.randint(10,30))
print("Each machine's working time:",Time_pcs)

Each machine's working time: [16, 25, 25]


In [7]:
#The times of the appearences of each machine
m_frequency=[]  
for i in range(Machine_num):
    m_frequency.append(0)
for i in range(len(S)):
    m_frequency[S[i]]=m_frequency[S[i]]+1
print("Machine appearence times:",m_frequency)

Machine appearence times: [34, 20, 30]


In [8]:
# Total processing time for each type of machine
Total_m_time=[]
for i in range(Machine_num):
    Total_m_time.append(m_frequency[i]*Time_pcs[i])
print("Total processing time for each type of machine:\n", Total_m_time)

Total processing time for each type of machine:
 [544, 500, 750]


In [9]:
min_time = Total_m_time[0]
for i in range(len(Total_m_time)):
    if (min_time>Total_m_time[i]):
        min_time = Total_m_time[i]
min_time

500

In [10]:
# to average the finishing time, attribute some assistant machines for the certain type
m_added=[]
for i in range(Machine_num):
    m_added.append(round(Total_m_time[i]/min_time))
print("Numbers of machines in each type:\n", m_added)

Numbers of machines in each type:
 [1, 1, 2]


# Parameter Assignment 

In [11]:
global block1#The amount of products that have been blocked by other products
block1=0
global block2  #How many products are blocked by other products
block2=0

In [12]:
global block_time
block_time=0

In [13]:
global m_vacant
m_vacant=Machine_num

In [14]:
global m_occupied
m_occupied=0

In [15]:
track_time=[]
track_p_blockednum=[]   #The number of products that are blocked at a certain time

In [16]:
# recording machines of each type
n_machine_each_type=[]
for i in range(0,Machine_num):
    n_machine_each_type.append(1)
n_machine_each_type

[1, 1, 1]

In [17]:
# wait list in each type of machine
wait=[]
for i in range(Machine_num):
    wait.append([])
wait

[[], [], []]

# Simpy Environment

In [63]:
env = simpy.Environment()

In [64]:
#set the number of machines of each type as resources
res=[]
for i in range(Machine_num):
    res.append(simpy.Resource(env,capacity=m_added[i]))

In [65]:
res

# Simulation Process 

In [66]:
def sys_process(product_num,i,res,env):
    global block1
    global block2
    global block_time
    global m_vacant
    global m_occupied
    global block_time
    
    # request machine
    machine = product_num
    print(f'Current time is {env.now}: NO.{i}, Product type {product_num} requests for machine{machine}')
    wait[machine].append(product_num) #add arriving products into the waiting list 
    t_request=env.now
    block2+=1
    
    #go into the machine
    with res[machine].request() as req:
        yield req
        m_occupied=m_occupied+1
        m_vacant=m_vacant-1
        t_in=env.now
        wait[machine].remove(product_num)
        block2=block2-1
        if (t_request!=t_in):
            block1=block1+1
            block_time=block_time+t_in-t_request
        print(f'Current time is {env.now}: NO.{i}, Product type {product_num} goes into the machine{machine}')
        yield env.timeout(Time_pcs[machine])
        m_vacant=m_vacant+1
        m_occupied=m_occupied-1
        print(f'Current time is {env.now}: NO.{i},Product type {product_num} goes out the machine{machine}')


In [67]:
def input_sequence(res,env):
    for i in range(inpro_seq+1):
        #the time gap between the input sequence is random
        yield env.timeout(np.random.randint(10,15))
        print(f'Current time is {env.now}: NO.{i}, Product type {S[i]} goes into the system')
        env.process(sys_process(S[i],i,res,env))

In [68]:
env.process(input_sequence(res,env))
env.run(until=1000)

Current time is 10: NO.0, Product type 2 goes into the system
Current time is 10: NO.0, Product type 2 requests for machine2
Current time is 10: NO.0, Product type 2 goes into the machine2
Current time is 22: NO.1, Product type 1 goes into the system
Current time is 22: NO.1, Product type 1 requests for machine1
Current time is 22: NO.1, Product type 1 goes into the machine1
Current time is 34: NO.2, Product type 2 goes into the system
Current time is 34: NO.2, Product type 2 requests for machine2
Current time is 34: NO.2, Product type 2 goes into the machine2
Current time is 35: NO.0,Product type 2 goes out the machine2
Current time is 45: NO.3, Product type 2 goes into the system
Current time is 45: NO.3, Product type 2 requests for machine2
Current time is 45: NO.3, Product type 2 goes into the machine2
Current time is 47: NO.1,Product type 1 goes out the machine1
Current time is 55: NO.4, Product type 0 goes into the system
Current time is 55: NO.4, Product type 0 requests for mach

In [69]:
block_time

29658